#### Segmenting and Clustering Neighborhoods in Toronto : last part (3)

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import io

#### Create table - step 1 use BeautifulSoup to scrape data from website:

In [4]:
wiki_url= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(wiki_url)

#Generate data table, leads to dataframe creation: find table in scraped content
created_table = soup.find('table',{'class':'wikitable sortable'})
created_table;

rows_data = []
for row in created_table.find_all("tr"):
    col = row.find_all("td")
    col = [ele.text.strip() for ele in col]
    rows_data.append(col)

rows_data;

df_1 = pd.DataFrame(rows_data)
df_1 = df_1.rename(columns={0:"Postcode",1:"Borough",2:"Neighborhood"})
df_1.head()

#drop 'Bourough' = 'Not assigned'
df_2 = df_1.copy()
df_2 = df_1.drop([0])
df_2 = df_2.drop(df_2[df_2['Borough']=='Not assigned'].index)
df_2 = df_2.reset_index(drop=True)
df_2.head();

# Find Neighborhood' = 'Not assigned'
df_2.loc[6];

#Replace values where not assigned
df_3 = df_2.copy()

df_3['Neighborhood'] = df_3.apply(lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1 )

df_3.loc[6];

df_4 = df_3.copy()
df_4 = (df_4.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())
df_4 = pd.DataFrame(df_4)
df_4.head()

#Shape of dataframe 4
#df_4.shape

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"West Hill,Guildwood,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Part 2

In [5]:
csv_data_url="http://cocl.us/Geospatial_data"
source=requests.get(csv_data_url).content
df_format=pd.read_csv(io.StringIO(source.decode('utf-8')))

df_format.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(df_format, df_4, on='Postcode')

df = df[['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()
#df.shape

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#####                            Part 3 begins after combining 1st and 2bd parts

In [6]:
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

In [7]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))
#df = df_table_final

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [8]:
df_toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [10]:
!pip install folium
import folium 

    100% |████████████████████████████████| 92kB 3.2MB/s ta 0:00:01
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/l

##### Toronto map and neighbourhoods

In [11]:
# Toronto map with lat and long values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# Including markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
  label = '{},{}'.format(neighborhood,borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

##### Use of Fourssquare API with my credentials to start exploring

In [12]:
CLIENT_ID = 'G2XVYHHSYR04AAMJH5CE5WOJHCYM1PLZXA41N1T3L45SEVUV' # my Foursquare ID
CLIENT_SECRET = 'WK04KW5NJNN2FY1SZCHW03LQCSDTP5WQVXUKFT3S3J154FA1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G2XVYHHSYR04AAMJH5CE5WOJHCYM1PLZXA41N1T3L45SEVUV
CLIENT_SECRET:WK04KW5NJNN2FY1SZCHW03LQCSDTP5WQVXUKFT3S3J154FA1


##### Exploring lawrence park

In [14]:
#Lawrence Park index[4]
neighborhood_latitude = df_toronto.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[4, 'Longitude'] # neighborhood longitude value
neighborhood_name = df_toronto.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


##### Top 100 venues in the park and in 500m radius

In [15]:
#GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=G2XVYHHSYR04AAMJH5CE5WOJHCYM1PLZXA41N1T3L45SEVUV&client_secret=WK04KW5NJNN2FY1SZCHW03LQCSDTP5WQVXUKFT3S3J154FA1&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8a18919fb6b70a24b9ed8d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50e6da19e4b0d8a78a0e9794-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '50e6da19e4b0d8a78a0e9794',
       'location': {'address': '3055 Yonge Street',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Lawrence Avenue East',
        'distance': 465,
        'formattedAddress': ['3055 Yonge Street (Lawrence Avenue East)',
         'Toronto ON',
         'Canada'],
        'labeledLatLngs': [{

##### Clean json file and transform into pd dataframe

In [17]:
# get the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 

# filter cols
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean cols
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [20]:
park_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=17)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(park_map) 
    
park_map

##### Clustering of venues over the neighbourhood

In [21]:
#Get all venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#run function for all toronto neighborhoods and create df 'toronto_venues'
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
Riverdale,The Danforth West
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Rathnelly,Summerhill West,Deer Park,South Hill,Forest Hill SE
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Regent Park,Harbourfront
Garden District,Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Union Station,Harbourfront East,Toronto Islands
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
North Midtown,Yorkville,The Annex
University of Toronto,Harbord
Chinatown,Kensington Market,Grange Park
King and Spadina,Harbourfront West,Island airport,Railway Lands,South Niagara,Bathurst Quay,CN Tower
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Roncesvalles,Parkda

In [26]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale,The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [27]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"Cabbagetown,St. James Town",45,45,45,45,45,45


In [28]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 unique categories.


##### Perfome one-hot encoding on venue category

In [29]:
toronto_encoding = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood col back to dataframe
toronto_encoding['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_encoding.columns[-1]] + list(toronto_encoding.columns[:-1])
toronto_encoding = toronto_encoding[fixed_columns]

toronto_encoding.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_grouped = toronto_encoding.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,Richmond,King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Central Bay Street,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.000000,0.00000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000,0.000000,0.00
6,"Chinatown,Kensington Market,Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010204,0.00000,0.00,0.051020,0.000000,0.051020,0.010204,0.000000,0.000000,0.00
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Church and Wellesley,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.000000,0.00000,0.00,0.000000,0.011765,0.011765,0.000000,0.011765,0.011765,0.00
9,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00


In [31]:
toronto_grouped.shape

(38, 236)

##### Five most common venues next to each neighbourhood

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,Richmond,King----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.03
3      Farmers Market  0.03
4  Seafood Restaurant  0.03


----Brockton,Exhibition Place,Parkdale Village----
               venue  freq
0        Coffee Shop  0.09
1     Breakfast Spot  0.09
2               Café  0.09
3  Convenience Store  0.05
4      Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1  Auto Workshop  0.06
2           Park  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.07
2          Pub  0.04
3       Market  0.04
4         Café  0.04


----Central Bay Street----
                venue  f

##### Sort ascending order function

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Café,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Pet Store,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Moving Target
4,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Pub,Bakery
5,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bubble Tea Shop
6,"Chinatown,Kensington Market,Grange Park",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Coffee Shop
7,Christie,Grocery Store,Café,Park,Athletics & Sports,Baby Store
8,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Restaurant,Sushi Restaurant
9,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant


##### Neighbourhoods clustering

In [36]:
# set number of clusters
kclusters = 15

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([ 8,  8,  8, 14,  8,  8,  0, 12,  0,  8,  0, 10,  0,  8,  5], dtype=int32)

In [37]:
toronto_combined = df_toronto

# add clustering labels
toronto_combined['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_combined = toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_combined.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,8,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Falafel Restaurant
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,8,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant
2,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572,8,Sandwich Place,Board Shop,Ice Cream Shop,Brewery,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,14,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,8,Park,Swim School,Bus Line,Women's Store,Diner


In [39]:
toronto_combined.tail()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,0,Breakfast Spot,Café,Coffee Shop,Pet Store,Bar
34,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,6,Café,Mexican Restaurant,Grocery Store,Arts & Crafts Store,Sandwich Place
35,M6R,West Toronto,"Roncesvalles,Parkdale",43.648960,-79.456325,8,Breakfast Spot,Gift Shop,Bookstore,Eastern European Restaurant,Restaurant
36,M6S,West Toronto,"Swansea,Runnymede",43.651571,-79.484450,8,Café,Coffee Shop,Pizza Place,Italian Restaurant,Sushi Restaurant
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,8,Yoga Studio,Auto Workshop,Park,Pizza Place,Moving Target


##### Venues clustered by map-to conclude the task

In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_combined['Latitude'], toronto_combined['Longitude'], toronto_combined['Neighborhood'], toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters